In [85]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

from pandas_summary import DataFrameSummary
from pathlib import Path

import os, math
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

pd.set_option('display.max_columns', 1000)
pd.set_option('display.max_rows', 400)
sns.set()
os.chdir('/home/krivas/projects/analysis-project/')
from src.utils import convertInt, convertDate, add_datediffs, add_dayscount, add_datefeatures,\
                        make_set, run_model, apply_cats, get_week_labels


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [37]:
DATA        = Path('data')
RAW         = DATA/'raw'
PROCESSED   = DATA/'processed'

# Reading data

In [38]:
csv = pd.read_csv(PROCESSED/'HistoricoJob.csv')

In [39]:
csv.columns

Index(['Id_HistoricoJob', 'Id_Job', 'Id_Malla', 'Fecha_Carga_Scheduler',
       'Fecha_Ejec_Inicio', 'Fecha_Ejec_Fin', 'Duracion', 'Promedio', 'Agente',
       'Mxrc', 'Maxcmpc', 'Grupo', 'Force_Complete', 'CCF', 'Estado',
       'duracion_int', 'promedio_int', 'Fecha_Ejec_Inicio_Int',
       'Hora_Ejec_Inicio_Int', 'Fecha_Ejec_Fin_Int', 'Hora_Ejec_Fin_Int'],
      dtype='object')

In [80]:
target_date = 20180515
n_weeks = 10

In [81]:
week_labels = get_week_labels(n_weeks, 2018, 5, 14)
week_labels

[20180305,
 20180312,
 20180319,
 20180326,
 20180402,
 20180409,
 20180416,
 20180423,
 20180430,
 20180507,
 20180514]

In [82]:
%%time
weeks = [make_set(csv[csv.Fecha_Ejec_Inicio_Int.apply(lambda x: w1<=x<w2)], 
                    csv.loc[csv['Fecha_Ejec_Inicio_Int'] == target_date]) for w1, w2 in zip(week_labels[:-1], week_labels[1:])]

CPU times: user 1min 27s, sys: 448 ms, total: 1min 28s
Wall time: 1min 28s


In [83]:
weeks.append(make_set(csv.loc[csv['Fecha_Ejec_Inicio_Int'] == target_date], csv.loc[csv['Fecha_Ejec_Inicio_Int'] == target_date]))

# Preprocessing

In [87]:
%%time
for temp in weeks:
    add_datediffs(temp, csv)
    add_dayscount(temp, csv)
    add_datefeatures(temp)

CPU times: user 13.9 s, sys: 0 ns, total: 13.9 s
Wall time: 13.9 s


In [90]:
weeks[1].head()

,Fecha_Ejec_Inicio_Int,Id_Job,Id_Malla,duracion_int,DaysSinceMainframeOp,DaysCountMainframeOp,DiaSemana
0,20180312.0,@AK2ZF29,02FBFCL2,1.0,205,1,Monday
1,20180312.0,@D2BYJQE,02DBWNH2,464.0,254,0,Monday
2,20180312.0,@D2BYVLB,02DBWNH2,961.0,128,1,Monday
3,20180312.0,@D2MKV99,02MKH993,5.0,254,0,Monday
4,20180312.0,@D2MKV99,04MKH993,2.0,254,0,Monday


In [91]:
cat_cols = ['DiaSemana']

In [93]:
%%time
for w in weeks:
    for col in cat_cols:
        w[col] = w[col].astype('category').cat.codes

CPU times: user 36 ms, sys: 0 ns, total: 36 ms
Wall time: 32.9 ms


# Model

In [94]:
#from src.utils import run_model
from lightgbm import LGBMRegressor

In [95]:
target_col = 'duracion_int'
pred_col = 'Prediccion'
id_cols = ['Fecha_Ejec_Inicio_Int', 'Id_Job', 'Id_Malla']
metric_names = ['mse']

In [96]:
weeks[len(week_labels) - 1][pred_col] = 0

In [98]:
%%time
results = None
output = []
for i, w in enumerate(weeks[1:]):
    train, val, test = pd.concat(weeks[:i+1], axis=0), w, weeks[-1]
    X_train, y_train = train.drop(id_cols + [target_col], axis=1), \
                       train[target_col]
    if pred_col in val.columns: # when test acts as validation
        X_val, y_val = None, None
    else:
        X_val, y_val = val.drop(id_cols + [target_col], axis=1), \
                       val[target_col]
    X_test = test.drop(id_cols + [target_col, pred_col], axis=1)
    y_test, _, results, model = run_model(
            LGBMRegressor(n_estimators=120),
            X_train, y_train, X_val, y_val, X_test,
            metric_names, results,
            early_stopping=True)
    output.append([y_test, model])

[1]	valid_0's l2: 9.07478e+07
Training until validation scores don't improve for 30 rounds.
[2]	valid_0's l2: 9.07987e+07
[3]	valid_0's l2: 9.08671e+07
[4]	valid_0's l2: 9.09458e+07
[5]	valid_0's l2: 9.1033e+07
[6]	valid_0's l2: 9.11151e+07
[7]	valid_0's l2: 9.11108e+07
[8]	valid_0's l2: 9.11089e+07
[9]	valid_0's l2: 9.11773e+07
[10]	valid_0's l2: 9.12467e+07
[11]	valid_0's l2: 9.12386e+07
[12]	valid_0's l2: 9.12306e+07
[13]	valid_0's l2: 9.12212e+07
[14]	valid_0's l2: 9.12115e+07
[15]	valid_0's l2: 9.12571e+07
[16]	valid_0's l2: 9.13067e+07
[17]	valid_0's l2: 9.13087e+07
[18]	valid_0's l2: 9.13082e+07
[19]	valid_0's l2: 9.11997e+07
[20]	valid_0's l2: 9.1113e+07
[21]	valid_0's l2: 9.11413e+07
[22]	valid_0's l2: 9.11655e+07
[23]	valid_0's l2: 9.11642e+07
[24]	valid_0's l2: 9.11915e+07
[25]	valid_0's l2: 9.11114e+07
[26]	valid_0's l2: 9.11308e+07
[27]	valid_0's l2: 9.11526e+07
[28]	valid_0's l2: 9.1154e+07
[29]	valid_0's l2: 9.11576e+07
[30]	valid_0's l2: 9.11788e+07
[31]	valid_0's l2: 9

In [99]:
model.feature_importances_

array([2112, 1488,    0])

In [100]:
results

,params,time,trn_mse_mean,val_mse_mean
0,,10.04,8.789039e+07,9.074778e+07
1,,18.44,8.915758e+07,9.075653e+07
2,,28.49,8.950537e+07,7.770684e+07
3,,22.71,8.647963e+07,8.621906e+07
4,,15.07,8.640801e+07,9.106379e+07
5,,0.19,8.714620e+07,9.208557e+07
6,,0.17,8.780847e+07,9.020044e+07
7,,0.16,8.814984e+07,8.039051e+07
8,,0.23,8.713139e+07,9.907745e+07
9,,0.97,8.812255e+07,0.000000e+00


In [101]:
y_test = np.mean([x[0] for x in output], axis=0)

In [102]:
y_test

array([ 732.53004111, 1481.52059509, 1481.52059509, ..., 1481.52059509,
       1481.52059509, 1481.52059509])

In [103]:
from sklearn.metrics import mean_squared_error

In [105]:
mean_squared_error(weeks[-1:][0].duracion_int, y_test)

3860187.5033376487

In [298]:
mean_squared_error(days_data[-1:][0].duracion_int, y_test)

3466312.79488895